In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tabula
import dill
import re
import sys
from os import listdir
from os.path import isfile, join

In [67]:
mypath = '/home/joe/data/covid19/who/test'
#mypath = '/run/user/1000/gvfs/smb-share:server=majesty.local,share=filesjoe/data/who_covid19_reports/renamed'
filelist = '/home/joe/datavisions/covid19/filelist'


In [68]:
#dill.dump_session('data.db')
#dill.load_session('data.db')

In [69]:
def findFilesToProcess(mypath,filelist):
# find pdfs in path
# if the file has not been processed, process it

    # list of files in dir
    files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    dfFiles = pd.DataFrame(files, columns = ['file'])
    #print(dfFiles)
    
    # create df of done files
    doneFiles = open(filelist).read().splitlines()
    dfDone = pd.DataFrame(doneFiles, columns = ['file'])
    #print(dfDone)
    
    return dfFiles[dfFiles['file'].isin(dfDone['file']) == False]
    


In [101]:
# %load findChina.py
def findChina(tables):
# from the list of tables (dfs), find the one with china - return cumulative cases and deaths
    x = 'Province/' # id China with this
    y = 'Hubei'# id China with this
    output = ['N/A','N/A']
    for df in tables:
        if (x in df.columns.values):
            if (y in df[x].tolist()):
                output = re.split('\\s+',df['Cumulative'].iloc[-1])
    return output


In [103]:
def parsePdf(f):
    
    tables = tabula.read_pdf(f, pages = "all", multiple_tables = True)
    chinaNumbers = findChina(tables)
        
    return chinaNumbers
    

In [114]:
def processFile(mypath,filelist,f):
    date, ext = f.split('.')
    ffile = '/'.join([mypath,f])

    try:
        fl = open(filelist,"a+")
    except OSError:
        print("could not open file to append ", filelist)
        sys.exit()
    
    chinaNumbers = parsePdf(ffile)

    try:
        fl.write(f + "\n")
    except OSError:
        print("could not append ", filelist)
        sys.exit()

    try:
        fl.close()
    except OSError:
        print("could not close file list ", filelist)
        sys.exit()
        
    chinaNumbers.insert(0,'china')
    chinaNumbers.insert(0,date)
    
    return chinaNumbers

In [117]:
dfToProcess = findFilesToProcess(mypath,filelist)
rows, columns = dfToProcess.shape
#f = dfToProcess['file'].iloc[0]
#print(f)
if (rows > 0): #there are files to process, process the first file
    out = processFile(mypath,filelist,dfToProcess['file'].iloc[0])
    print(out)

Got stderr: Mar 10, 2020 4:19:33 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 10, 2020 4:19:34 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 10, 2020 4:19:34 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



['2020-03-06', 'china', '80711', '3045']
